# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [26]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [27]:
input_keyword = "沃德·坎寧安"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

沃德·坎寧安: %E6%B2%83%E5%BE%B7%C2%B7%E5%9D%8E%E5%AF%A7%E5%AE%89
/wiki/%E6%B2%83%E5%BE%B7%C2%B7%E5%9D%8E%E5%AF%A7%E5%AE%89


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [28]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


沃德·坎寧安（英語：Ward Cunningham，1949年5月26日－），是一名計算機程式設計師，也是Wiki概念的發明者。他也是設計模式和敏捷軟體方法的先驅之一。

他從普度大學獲得（電子工程和計算機科學的）交叉學科工學學士學位以及計算機科學的碩士學位。

他1995年在波特蘭模式知識庫創建了第一個Wiki站點。這個站點現在還在運作，致力於「人、項目和模式」，並且是一個「程序設計語言思想的非正式歷史」。例如，這個站點被用來為有用的軟體開放的模式語言以及極限編程的軟體方法的發展進行分類。坎寧安提到Wiki的概念是他在20世紀80年代末期想到的，並且他首先使用HyperCard堆疊的方法進行實現。

他是The Wiki Way（2001年）這本書的作者（與Bo Leuf合著）。

他是Cunningham & Cunningham, Inc.公司的創始人。他還是Wyatt Software研發部門的總裁，以及Tektronix Computer Research Laboratory的主要工程師。沃德·坎寧安之所以聞名的是對物件導向程序設計的開發實踐的貢獻，這個變種叫做極限程序設計，以及由他的WikiWikiWeb提供的社團。他還是「山坡組」的創始人，並且作為其贊助的「程序的模式語言」會議的程序主席。

2003年12月，沃德·坎寧安加入微軟，為微軟的「模式與實踐」組工作。

2005年10月，他轉入Eclipse基金會。

沃德·坎寧安現居俄勒岡州的波特蘭。



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [29]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [程式設計師] /wiki/%E7%A8%8B%E5%BA%8F%E5%91%98
外部連結: [Wiki] /wiki/Wiki
外部連結: [設計模式] /wiki/%E8%AE%BE%E8%AE%A1%E6%A8%A1%E5%BC%8F
外部連結: [敏捷軟體方法] /wiki/%E6%95%8F%E6%8D%B7%E5%BC%80%E5%8F%91
外部連結: [普度大學] /wiki/%E6%99%AE%E5%BA%A6%E5%A4%A7%E5%AD%B8
外部連結: [學士學位] /wiki/%E5%AD%A6%E5%A3%AB%E5%AD%A6%E4%BD%8D
外部連結: [碩士學位] /wiki/%E7%A1%95%E5%A3%AB%E5%AD%A6%E4%BD%8D
外部連結: [極限編程] /wiki/%E6%9E%81%E9%99%90%E7%BC%96%E7%A8%8B
外部連結: [HyperCard] /wiki/HyperCard
外部連結: [The Wiki Way] /wiki/The_Wiki_Way
外部連結: [物件導向程序設計] /wiki/%E9%9D%A2%E5%90%91%E5%AF%B9%E8%B1%A1%E7%A8%8B%E5%BA%8F%E8%AE%BE%E8%AE%A1
外部連結: [極限程序設計] /wiki/%E6%9E%81%E9%99%90%E7%A8%8B%E5%BA%8F%E8%AE%BE%E8%AE%A1
外部連結: [微軟] /wiki/%E5%BE%AE%E8%BD%AF
外部連結: [Eclipse基金會] /wiki/Eclipse%E5%9F%BA%E9%87%91%E4%BC%9A
外部連結: [俄勒岡州] /wiki/%E4%BF%84%E5%8B%92%E5%86%88%E5%B7%9E
外部連結: [波特蘭] /wiki/%E6%B3%A2%E7%89%B9%E5%85%B0_(%E4%BF%84%E5%8B%92%E5%86%88%E5%B7%9E)


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [43]:
import os
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        chk_dir = 'WikiArticle'
        if (not os.path.isdir(chk_dir)):
            os.mkdir(chk_dir)
         
        file = chk_dir+'/'+key_word.replace('ː','').replace('ˈ','').replace('/','')+'.txt'
        #print(key_word)
        with open(file, "w", encoding="utf-8") as f:
            for paragraph in content:
                f.write(paragraph.get_text())
        
        

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    #print(a_keyword,a_link)
                    external_link_dict[a_link]= a_keyword

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [44]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E6%B2%83%E5%BE%B7%C2%B7%E5%9D%8E%E5%AF%A7%E5%AE%89 (沃德·坎寧安)
遞迴層[1] - /wiki/%E7%A8%8B%E5%BA%8F%E5%91%98 (程式設計師)
遞迴層[2] - /wiki/%E7%A8%8B%E5%BA%8F%E8%AE%BE%E8%AE%A1 (程式設計)
遞迴層[2] - /wiki/%E8%BB%9F%E9%AB%94 (軟體)
遞迴層[2] - /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%A8%8B%E5%BA%8F (電腦程式)
遞迴層[2] - /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E8%AF%AD%E8%A8%80 (程式語言)
遞迴層[2] - /wiki/%E6%B1%87%E7%BC%96%E8%AF%AD%E8%A8%80 (組合語言)
遞迴層[2] - /wiki/COBOL (COBOL)
遞迴層[2] - /wiki/C%E8%AF%AD%E8%A8%80 (C)
遞迴層[2] - /wiki/C%2B%2B (C++)
遞迴層[2] - /wiki/C%E2%99%AF (C#)
遞迴層[2] - /wiki/JavaScript (JavaScript)
遞迴層[2] - /wiki/LISP (Lisp)
遞迴層[2] - /wiki/Python (Python)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E9%96%8B%E7%99%BC%E8%80%85 (Web開發人員)
遞迴層[2] - /wiki/%E6%B5%81%E5%8B%95%E6%87%89%E7%94%A8%E7%A8%8B%E5%BC%8F (行動應用程式)
遞迴層[2] - /wiki/%E9%9F%8C%E9%AB%94 (韌體)
遞迴層[2] - /wiki/%E8%BD%AF%E4%BB%B6%E5%B7%A5%E7%A8%8B%E5%B8%88 (軟體工程師)
遞迴層[2] - /wiki/%E9%9B%BB%E8%85%A6%E7%A7%91%E5%AD%B8%E5%AE%B6 (電腦科學家)
遞迴層[2] - /wiki/%E6%B8%B8%E6%